In [1]:
# make imports

In [2]:
# import datasets (disasters)

# Q1: Given a year is between 2000-2018, what is the chance that it's number of total disasters is greater than the average number of disasters for all years 1900-2018?